In [1]:
from manim import *
import math
import jupyter_capture_output

video_scene = " -v WARNING  --disable_caching double_slit_Scene"
image_scene = f" -v WARNING --disable_caching -r {2*427},{2*240}  -s double_slit_Scene"

Jupyter Capture Output v0.0.11


In [5]:
background_color = WHITE


# frequency over time
def nu_smooth(t):
	return 100 * (3 + math.erf(2/5*(t-21)))
	# return 400

In [100]:
class WaveSources(Mobject):
	def __init__(self, center_A = np.array([-2.5, -2, 0]), center_B = np.array([2.5, -2, 0]), c = 300, wave_width = 10, **kwargs):
		super().__init__(**kwargs)

		self.source_A = center_A                       # center of speaker A
		self.source_B = center_B                       # center of speaker B

		self.c = c                                      # speed of sound
		self.slit_width = 0.25
		self.r0 = self.slit_width                                    # minimal distance from the sound sources
		self.d = np.sqrt(sum((center_A-center_B)**2))        # distance between the sound sources
		self.circle_stroke_width = wave_width

		left_line = Line(start = center_A + 5*LEFT, end = center_A, color = BLACK, stroke_width = 20, buff = self.slit_width)
		mid_line = Line(start = center_A, end = center_B, color = BLACK, stroke_width = 20, buff = self.slit_width)
		right_line = Line(start = center_B, end = center_B + 5*RIGHT, color = BLACK, stroke_width = 20, buff = self.slit_width)

		left_line.z_index = 2
		mid_line.z_index = 2
		right_line.z_index = 2
		
		# low_half_space_rect = Polygon(
		#     center_A + 5*LEFT, center_B + 5*RIGHT, center_B + 5*RIGHT + 5*DOWN, center_A + 5*LEFT + 5*DOWN, 
		#     color = WHITE, fill_color = WHITE, fill_opacity = 1 
		#     )
		# low_half_space_rect.z_index = 1
		self.add(left_line, mid_line, right_line)



	# calculates the amplitude of the wave depending on frequency nu, distance from the source r, and time t
	def get_amplitude(self, nu, r, t):
		omega = 2*PI*nu                                 # circle frequency
		k = omega / self.c                              # wave number
		a0 = min(k * self.d / 4, k)             # a0 set to limit amplitude to 1
		return a0 / k / r * (np.cos(omega*t - k*r))
	
	
	# returns the wave field as a group of circles around both sound sources
	def get_waves(self, nu, t):
		# condition for waves: distance bigger than minimum, distance lower than speed of sound times time
		r_max = self.c * t                              # set r_max to maximum spread
		r = r_max                                     # set r to minimum spread              
		dr = self.c / nu                    # wavelength = radius increment 
		wave_group = VGroup()                           # group with all wave lines
		# radius lines
		while r > self.r0:
			amplitude = self.get_amplitude(nu, r, t)    # amplitude of the waves (between 0 and 1)
			# print(amplitude)
			wave_A = Arc(radius = r, start_angle = 0, angle = PI, arc_center = self.source_A, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude)
			wave_B = Arc(radius = r, start_angle = 0, angle = PI, arc_center = self.source_B, color = BLACK, stroke_width = self.circle_stroke_width, stroke_opacity = amplitude)
			wave_A.z_index = -5
			wave_B.z_index = -5
			wave_group.add(wave_A, wave_B)
			r -= dr

		# incoming wavefront
		if t < 0:
			wavefront_1_y_offset = (self.c * (t-self.r0/self.c)) + 0.1
			wavefront_2_y_offset = (self.c * (t-self.r0/self.c) - dr) + 0.1
			wavefront_3_y_offset = (self.c * (t-self.r0/self.c) - 2*dr) + 0.1
		else:
			wavefront_1_y_offset = (self.c * (t-self.r0/self.c)) % (-3*dr) + 0.1
			wavefront_2_y_offset = (self.c * (t-self.r0/self.c) - dr) % (-3*dr) + 0.1
			wavefront_3_y_offset = (self.c * (t-self.r0/self.c) - 2*dr) % (-3*dr) + 0.1
		wavefront_1 = Line(start = self.source_A + 5*LEFT + (wavefront_1_y_offset) * UP, end = self.source_B + 5*RIGHT + (wavefront_1_y_offset) * UP, color = BLACK, stroke_width = self.circle_stroke_width)
		wavefront_2 = Line(start = self.source_A + 5*LEFT + (wavefront_2_y_offset) * UP, end = self.source_B + 5*RIGHT + (wavefront_2_y_offset) * UP, color = BLACK, stroke_width = self.circle_stroke_width)
		wavefront_3 = Line(start = self.source_A + 5*LEFT + (wavefront_3_y_offset) * UP, end = self.source_B + 5*RIGHT + (wavefront_3_y_offset) * UP, color = BLACK, stroke_width = self.circle_stroke_width)  
		wavefront_1.z_index = 2
		wavefront_2.z_index = 2
		wavefront_3.z_index = 2
		wave_group.add(wavefront_1, wavefront_2, wavefront_3)
		return wave_group
	

	def get_intensity_function(self, nu, pattern_height):
		a = self.d                  # distance between the two slits
		b = 2 * self.slit_width     # slit width
		lam = self.c / nu           # wavelength
		y_dist = pattern_height - self.source_A[1] + 0.5

		def intensity_function(x):
			r = np.sqrt(x**2 + y_dist**2)
			alpha = a * x / r * PI / lam
			beta = b * x / r * PI / lam
			return (np.sin(beta) / beta)**2 * np.cos(alpha)**2
		return intensity_function
	

	def get_interference_legend(self, nu, opacity = 0.95):
		pattern_height = 3
		legend_bg = Rectangle(height = 4-pattern_height, width = 15, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = opacity).align_on_border(UP, buff = 0)
		legend_bg.z_index = 3
		ax_line = Line(start = [-8, pattern_height, 0], end = [8, pattern_height, 0], stroke_width = 2, color = BLACK)
		ax_line.z_index = 5

		ax = Axes(x_length = 15, y_length = 1, x_range = [-7.5, 7.5], y_range = [-1, 1]).move_to([0, pattern_height, 0]).set_color(BLACK)
		ax.z_index = 10
		intensity_function = self.get_intensity_function(nu, pattern_height)
		intensity_plot = ax.plot(intensity_function, color = RED)
		intensity_plot.z_index = 10
		return VGroup(legend_bg, ax_line, intensity_plot)



	# add a nu tracker as legend for the frequency
	def get_nu_legend(self, nu, opacity = 0.95):
		legend_bg = Rectangle(height = 1, width = 4, stroke_color = WHITE, stroke_opacity = 0, fill_opacity = opacity).align_on_border(RIGHT + UP, buff = 0.25).shift(1*DOWN)
		legend_bg.z_index = 3

		lam = self.c / nu           # wavelength
		nu_number_line = NumberLine(x_range = [0.5, 2, 0.5], length = 3, color = BLACK).move_to(legend_bg.get_center())
		for tick in nu_number_line.ticks:
			tick.z_index = 5
		nu_number_line.z_index = 5

		# calculate axis position
		# if type(nu) == int or type(nu) == float:
		x_number_line = np.log10(nu/2)                                  # position on the number line
		pos_number_line = nu_number_line.number_to_point(lam)
		nu_line = Line(start = pos_number_line+0.125*UP, end = pos_number_line-0.125*UP, color = RED, stroke_width = 2)
		nu_line.z_index = 10
		nu_text = Text(f"= {np.round(lam, 2)}", color = RED).scale(0.25).next_to(nu_line, 0.5*DOWN)
		lambda_text = Tex(r"$\lambda$", color = RED).scale(0.375).next_to(nu_text, LEFT).shift(0.18*RIGHT + 0.005*UP)
		for letter in lambda_text:
			letter[0].z_index = 5
		for letter in nu_text:
			letter.z_index = 5


		return VGroup(legend_bg, nu_number_line, nu_line, lambda_text, nu_text)

In [101]:
%%manim -qh --fps 60 $video_scene


class double_slit_Scene(Scene):
	def construct(self):
		self.camera.background_color = background_color

		# parameters
		c = 300
		t0 = -2
		nu = nu_smooth(t0)


		wave_source = WaveSources(c = c, wave_width = 15)
		self.add(wave_source)

		waves = wave_source.get_waves(nu, t0/c)
		self.add(waves)


		# legends
		interference_legend = wave_source.get_interference_legend(nu)
		# self.add(interference_legend)

		nu_legend = wave_source.get_nu_legend(nu)
		# self.add(nu_legend)


		def wave_updater(wave):
			t = time_tracker.get_value()
			nu_t = nu_smooth(t*c)
			wave.become(wave_source.get_waves(nu_t, t))


		def interference_legend_updater(legend):
			t = time_tracker.get_value()
			nu_t = nu_smooth(t*c)
			legend.become(wave_source.get_interference_legend(nu_t))


		def nu_legend_updater(legend):
			t = time_tracker.get_value()
			nu_t = nu_smooth(t*c)
			# print(t*c, nu_t)
			legend.become(wave_source.get_nu_legend(nu_t))


		time_tracker = ValueTracker(t0/c)
		waves.add_updater(wave_updater)
		
		self.play(time_tracker.animate.set_value(10/c), rate_func = linear, run_time = 12)
		self.play(time_tracker.animate.set_value(13/c), FadeIn(interference_legend), FadeIn(nu_legend), rate_func = linear, run_time = 3)

		nu_legend.add_updater(nu_legend_updater)
		interference_legend.add_updater(interference_legend_updater)

		self.play(time_tracker.animate.set_value(39/c), rate_func = linear, run_time = 23)

Manim Community v0.18.1

In [ ]:
# 'ffmpeg -f concat -i ndw_interference_merge_list.txt -c copy ndw_interference_F1.mp4'
# 'ffmpeg -f concat -i ndw_interference_bg_blue_merge_list.txt -c copy ndw_interference_bg_blue_F2.mp4'
# 'ffmpeg -f concat -i ndw_interference_hybrid_merge_list.txt -c copy ndw_interference_hybrid_F5.mp4'